In [ ]:
import sys, os, decimal, json
sys.path.append(os.path.realpath('..'))

from MiningBot.EveInterface.Interface import Interface
import json, time
import pyautogui
import socket


In [ ]:
config_dir = r'../MiningBot/Configs/configs.json'
config = json.load(open(config_dir))[socket.gethostname()]
game = Interface(config_dir=config_dir)

NameError: name 'Interface' is not defined

In [ ]:
def get_processed_cords(x, y):
    return x + config['monitor_offset_x'], y + config['monitor_offset_y']

In [ ]:
# existing station...

resource_map = {}

def perform_mapping():
    global resource_map
    targets = ['Site One Alpha', 'Site Two Alpha', 'Site Three Alpha']
    df = game.get_location_data(refresh_screen=True)
    print(df)
    for target in targets:
        try:
            print(f'Navigating to {target}')
            xy = df.loc[df['Name'] == target, 'click_target'].values[0]
            pyautogui.moveTo(xy)
            time.sleep(0.1)
            pyautogui.click(button='right')
            time.sleep(0.1)
            pyautogui.moveTo(xy[0] + 50, xy[1] + 25)
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(0.1)
            pyautogui.moveTo(1, 1)
            time.sleep(60)

            print('scanning...')
            pyautogui.moveTo(get_processed_cords(config['scanner_button_x'], config['scanner_button_y']))
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(7)
            resource_map[target] = game.get_survey_scan_data(refresh_screen=True, extract_type='bool')
            print(f'Data Collected!')
        except Exception as e:
            print(e)

In [ ]:
mining_sites = config['mining_sites']
def find_Mining_spot(keep_finding = True):
    location_df = game.get_location_data(refresh_screen=True)
    while True:
        for target in mining_sites:
            print(f'Navigating to {target}')
            xy = location_df.loc[location_df['Name'] == target, 'click_target'].values[0]
            pyautogui.moveTo(xy)
            time.sleep(0.1)
            pyautogui.click(button='right')
            time.sleep(0.1)
            pyautogui.moveTo(xy[0] + 50, xy[1] + 25)
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(0.1)
            pyautogui.moveTo(1, 1)
            time.sleep(60)

            print('scanning...')
            pyautogui.moveTo(get_processed_cords(*config['scanner_button_target']))
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(7)
            scan_df = game.get_survey_scan_data(refresh_screen=True, extract_type='bool')
            if len(scan_df[scan_df['Quantity']==True]) >= 2:
                return target
        if keep_finding == False:
            print('keep_finding = False, sending Home.')
            target = 'Home'
            print(f'Navigating to {target}')
            xy = location_df.loc[location_df['Name'] == target, 'click_target'].values[0]
            pyautogui.moveTo(xy)
            time.sleep(0.1)
            pyautogui.click(button='right')
            time.sleep(0.1)
            pyautogui.moveTo(xy[0] + 50, xy[1] + 25)
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(0.1)
            pyautogui.moveTo(1, 1)
            time.sleep(60)
            print('Unable to locate Ore, Done...')
            return    

In [ ]:
#game.get_survey_scan_data(refresh_screen=True, extract_type='bool')

In [ ]:
#find_Mining_spot()

In [ ]:
def mine_till_full():
    field_depleted = False
    while True:
        cargo_percent = game.get_cargo_data(refresh_screen=True)
        print(f'Cargo {cargo_percent:.2f}')
        if cargo_percent > 0.9 or field_depleted:
            target = 'Home'
            location_df = game.get_location_data(refresh_screen=True)
            print(f'Navigating to {target}')
            xy = location_df.loc[location_df['Name'] == target, 'click_target'].values[0]
            pyautogui.moveTo(xy)
            time.sleep(0.1)
            pyautogui.click(button='right')
            time.sleep(0.1)
            pyautogui.moveTo(xy[0] + 50, xy[1] + 25)
            time.sleep(0.1)
            pyautogui.click(button='left')
            time.sleep(0.1)
            pyautogui.moveTo(10, 10)
            time.sleep(180) # warning, docking can take some time.
            break
        scan_df = game.get_survey_scan_data(refresh_screen=True, extract_type='bool')
        scan_df = scan_df[scan_df['Quantity'] == True]
        
        if len(scan_df) < 2:
            field_depleted = True
        
        snap_df = scan_df[0:2]
        snap_df = snap_df[~snap_df['Locked'] == True]
        indicies = snap_df.index
        if len(indicies) == 2:
            print('starting 2x...')
            for i in indicies:
                pyautogui.moveTo(scan_df.loc[i, 'click_target'])
                time.sleep(0.1)
                pyautogui.keyDown('ctrl')
                time.sleep(0.1)
                pyautogui.click()
                time.sleep(0.1)
                pyautogui.keyUp('ctrl')
                time.sleep(1)
            time.sleep(4)
            xy=None
            for i, index in enumerate(indicies):
                xy = scan_df.loc[index, 'click_target']
                pyautogui.moveTo(xy)
                time.sleep(0.1)
                pyautogui.click()
                time.sleep(0.1)
                pyautogui.press(f'f{i + 1}')
                time.sleep(1)
            xy = (10, 10)
            pyautogui.moveTo(xy)
            print('started...')
            time.sleep(175)    
        else:
            print('skipping...')
            time.sleep(30)           

In [ ]:
def unload():
    pyautogui.moveTo(get_processed_cords(*config['click_and_drag_inv_box'][2:4])) #
    time.sleep(0.1)
    x, y = get_processed_cords(*config['click_and_drag_inv_box'][0:2])
    pyautogui.dragTo(x, y, 1,  button='left')
    time.sleep(0.1)
    pyautogui.moveTo(get_processed_cords(*config['click_and_drag_inv_line'][0:2]))
    time.sleep(0.1)
    x, y = get_processed_cords(*config['click_and_drag_inv_line'][2:4])
    pyautogui.dragTo(x, y, 1,  button='left')
    time.sleep(0.1)
    pyautogui.moveTo(get_processed_cords(*config['exit_hanger_target']))
    time.sleep(0.1)
    pyautogui.click(button='left')

    time.sleep(60)

In [ ]:
while True:
    try:
        find_Mining_spot()
        mine_till_full()
        unload()
    except Exception as e:
        print(e)
        pass